In [1]:
from pathlib import Path
import sys  
import os
import pandas as pd 
from datetime import datetime
import scipy.sparse as sps
from numpy import linalg as LA

import numpy as np
from operator import itemgetter
import optuna as op

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM

from scipy.sparse import hstack, vstack
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [5]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [6]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 168 ( 0.5%) Users that have less than 1 test interactions


In [7]:
from libs.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

In [9]:
ItemKNNCFRecommender(URM_train).fit(similarity="asymmetric")

Similarity column 38121 (100.0%), 4609.88 column/sec. Elapsed time 8.27 sec


In [8]:
def objective_function_KNN(optuna_trial):
                       
        
    recommender_instance = ItemKNNCFRecommender(URM_train)
    
    similarity = 'asymmetric'
    
    full_hyperp = {"similarity": similarity,
                   "topK": optuna_trial.suggest_int("topK", 2, 100),
                   "shrink": optuna_trial.suggest_int("shrink", 1 , 50),
                   "feature_weighting": optuna_trial.suggest_categorical("feature_weighting", ["BM25", "TF-IDF", "none"])
                  }
    
    if similarity == "asymmetric":
        full_hyperp["asymmetric_alpha"] = optuna_trial.suggest_float("asymmetric_alpha", 0, 2, log=False)
        full_hyperp["normalize"] = True

    elif similarity == "tversky":
        full_hyperp["tversky_alpha"] = optuna_trial.suggest_float("tversky_alpha", 0.1, 0.99, log=False)
        full_hyperp["tversky_beta"] = optuna_trial.suggest_float("tversky_beta", 0.1, 1.9, log=False)
        full_hyperp["normalize"] = True 

    elif similarity == "euclidean":
        full_hyperp["normalize_avg_row"] = optuna_trial.suggest_categorical("normalize_avg_row", [True, False])
        full_hyperp["similarity_from_distance_mode"] = optuna_trial.suggest_categorical("similarity_from_distance_mode", ["lin", "log", "exp"])
        full_hyperp["normalize"] = optuna_trial.suggest_categorical("normalize", [True, False])
        
    
    recommender_instance.fit(**full_hyperp)
    
    eval_res, _ = evaluator_validation.evaluateRecommender(recommender_instance)    
    return eval_res["MAP"][10]

In [9]:
class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame()
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        
        self.results_df = self.results_df._append(hyperparam_dict, ignore_index=True)

In [ ]:
optuna_study = op.create_study(direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function_KNN,
                      callbacks=[save_results],
                      n_trials = 250)

[I 2024-12-10 22:35:51,844] A new study created in memory with name: no-name-c58e2f81-95a1-4a3b-a51e-ae6d3bb7c8c2


Similarity column 38121 (100.0%), 4783.27 column/sec. Elapsed time 7.97 sec
EvaluatorHoldout: Processed 35568 (100.0%) in 10.59 sec. Users per second: 3357


[I 2024-12-10 22:36:10,476] Trial 0 finished with value: 0.03958238277317092 and parameters: {'topK': 16, 'shrink': 27, 'feature_weighting': 'none', 'asymmetric_alpha': 0.9838746470798099}. Best is trial 0 with value: 0.03958238277317092.


Similarity column 38121 (100.0%), 4800.97 column/sec. Elapsed time 7.94 sec
EvaluatorHoldout: Processed 35568 (100.0%) in 11.25 sec. Users per second: 3162


[I 2024-12-10 22:36:29,705] Trial 1 finished with value: 0.029091866953708544 and parameters: {'topK': 16, 'shrink': 42, 'feature_weighting': 'none', 'asymmetric_alpha': 1.6420970121958254}. Best is trial 0 with value: 0.03958238277317092.


...

[I 2024-12-10 23:50:35,412] Trial 210 finished with value: 0.05457711229640892 and parameters: {'topK': 5, 'shrink': 24, 'feature_weighting': 'none', 'asymmetric_alpha': 0.07559336249462655}. Best is trial 151 with value: 0.054922145552626186.


Similarity column 38121 (100.0%), 4648.83 column/sec. Elapsed time 8.20 sec
EvaluatorHoldout: Processed 35568 (100.0%) in 8.77 sec. Users per second: 4056


[I 2024-12-10 23:50:52,435] Trial 211 finished with value: 0.05462988177352037 and parameters: {'topK': 5, 'shrink': 24, 'feature_weighting': 'none', 'asymmetric_alpha': 0.0682166643933744}. Best is trial 151 with value: 0.054922145552626186.


Similarity column 38121 (100.0%), 4681.76 column/sec. Elapsed time 8.14 sec
EvaluatorHoldout: Processed 35568 (100.0%) in 8.90 sec. Users per second: 3994


[I 2024-12-10 23:51:09,530] Trial 212 finished with value: 0.05431543714164176 and parameters: {'topK': 6, 'shrink': 24, 'feature_weighting': 'none', 'asymmetric_alpha': 0.11883244920556446}. Best is trial 151 with value: 0.054922145552626186.
